In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data.dataloader as dataloader
import torch.optim as optim

from torch.autograd import Variable
from torchvision import transforms
from torchvision.datasets import MNIST

## Data

* [Pytorch Transform Documentation](http://pytorch.org/docs/torchvision/transforms.html)


1. **torchvision.transforms.Compose:** 여러개의 tranforms을 실행합니다. 
2. **torchvision.transforms.ToTensor:** PIL.Image 또는 [0, 255] range의 Numpy array(H x W x C)를 (C x H x W)의 **[0.0, 1.0] range**를 갖은 torch.FloatTensor로 변형시킵니다. <br>여기서 포인트가 0에서 1사이의 값을 갖은 값으로 normalization이 포함되있습니다. 
3. **dataloader.DataLoader:** 사용하여 training시킬때 1개의 batch를 가져올때 shape이 **torch.Size([64, 1, 28, 28])** 이렇게 나옵니다. 

In [2]:
train = MNIST('./data', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

test = MNIST('./data', train=False, download=True, transform=transforms.Compose([
    transforms.ToTensor(), # ToTensor does min-max normalization. 
]), )

train_loader = dataloader.DataLoader(train, shuffle=True, batch_size=64, num_workers=1, pin_memory=True)
test_loader = dataloader.DataLoader(test, shuffle=True, batch_size=64, num_workers=1, pin_memory=True)

In [3]:
train_data = train.train_data
train_data = train.transform(train_data.cpu().numpy())

print('[Train]')
print(' - Numpy Shape:', train.train_data.cpu().numpy().shape)
print(' - Tensor Shape:', train.train_data.size())
print(' - Transformed Shape:', train_data.size())
print(' - min:', torch.min(train_data))
print(' - max:', torch.max(train_data))
print(' - mean:', torch.mean(train_data))
print(' - std:', torch.std(train_data))

[Train]
 - Numpy Shape: (60000, 28, 28)
 - Tensor Shape: torch.Size([60000, 28, 28])
 - Transformed Shape: torch.Size([28, 60000, 28])
 - min: 0.0
 - max: 1.0
 - mean: 0.13066047740240005
 - std: 0.3081078089011192


## Model

In [4]:
class Model(nn.Module):
    def __init__(self, parameters=[784, 625, 361, 144, 49, 1]):
        super(Model, self).__init__()
        
        self.fc1 = nn.Linear(784, 625)
        self.bc1 = nn.BatchNorm1d(625)
        
        self.fc2 = nn.Linear(625, 361)
        self.bc2 = nn.BatchNorm1d(361)
        
        self.fc3 = nn.Linear(361, 144)
        self.bc3 = nn.BatchNorm1d(144)
        
        self.fc4 = nn.Linear(144, 49)
        self.bc4 = nn.BatchNorm1d(49)
        
        self.fc5 = nn.Linear(49, 1)
        
    def forward(self, x):
        print(x.size(), 'ahah1')
        x = x.view((-1, 784))
        h = self.fc1(x)
        h = nn.ReLU(self.bc1(h))
        
        h = self.fc2(x)
        h = nn.ReLU(self.bc2(h))
        
        h = self.fc3(x)
        h = nn.ReLU(self.bc3(h))
        
        h = self.fc4(x)
        h = nn.ReLU(self.bc4(h))
        
        h = self.fc5(x)
        out = nn.CrossEntropyLoss(h)
        return out

model = Model()
model.cuda() # CUDA!

optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train

In [6]:
model.train()
for data, target in train_loader:
    data = Variable(data.cuda())
    
    
    optimizer.zero_grad()
    output = model(data)
    
    
    break

torch.Size([64, 1, 28, 28]) ahah1


RuntimeError: size mismatch at /home/anderson/Downloads/pytorch/torch/lib/THC/generic/THCTensorMathBlas.cu:243